In [23]:


from io import BytesIO
from base64 import b64decode

from IPython.core import magic_arguments
from IPython.core.magic import Magics, cell_magic, magics_class
from IPython.display import display
from IPython.utils.capture import capture_output
from PIL import Image
from pathlib import Path
from pprint import pprint # for debugging
import re


def path_preprocessing(path):
    # print(type(path))  # for debugging
    path_pathlib = Path(path)
    if not path_pathlib.parent.exists():
        path_pathlib.parent.mkdir(exist_ok=False, parents=True)
        print(f"Note: The {path_pathlib.parent} directory was successfully created.")

    if not path_pathlib.exists():
        print(f"Output saved by creating file at {path_pathlib}.")
        # TODO: Call this later maybe?

    if path_pathlib.exists():
        print(f"Output saved by overwring previous file at {path_pathlib}.")
        # TODO: Call this later maybe?

    return path_pathlib

@magics_class
class CaptureMagic(Magics):
    @magic_arguments.magic_arguments()  ################### TEXT
    @magic_arguments.argument(
        "--path",
        "-p",
        default=None,
        help=("The path where the text will be saved to"),
    )
    @cell_magic
    def to_md(self, line, cell):
        args = magic_arguments.parse_argstring(CaptureMagic.to_md, line)
        paths_string = args.path.strip('"').split(" ")
        paths_pathlib = []
        for path_str in paths_string:
            path_pathlib = path_preprocessing(path_str)
            paths_pathlib.append(path_pathlib)

        # TODO: for capture_text, paths_string has only one element, so this would not be necessary here.
        # It's just for convenicene, as the other capture functions are implemented with the same loop.

        with capture_output(stdout=True, stderr=False, display=False) as result:
            self.shell.run_cell(cell)
            message = result.stdout

        if len(message) == 0:
            raise ValueError("No standard output (stdout) found!")
        print(message)
        dest = paths_pathlib[0]
        dest.write_text(message)
        print("#### Markdown #### \n")
        markdown_string = ""
        markdown_string += "```python\n"
        markdown_string += cell
        markdown_string += "```\n"
        markdown_string += "```\n"
        markdown_string += "# out:\n"

        markdown_string += str(result)
        markdown_string += "```\n"
        print(markdown_string)



ipy = get_ipython()
ipy.register_magics(CaptureMagic)

In [24]:
%%to_md  --path "test.txt"
print("hi")
x =10
x

10

Output saved by overwring previous file at test.txt.
hi

#### Markdown #### 

```python
print("hi")
x =10
x
```
```
# out:
hi
```

